# Creating a Custom Data Source

By default, **PyBroker** provides [DataSources](https://pybroker.com/en/latest/reference/pybroker.data.html#pybroker.data.DataSource) for [Yahoo Finance](https://finance.yahoo.com/) and [Alpaca](https://alpaca.markets/). However, it is also possible to write your own ```DataSource```. The example below implements a ```DataSource``` that loads data from a CSV file:

In [1]:
import pandas as pd
import pybroker
from pybroker.data import DataSource

class CSVDataSource(DataSource):
    
    def __init__(self):
        super().__init__()
        # Register custom columns in the CSV.
        pybroker.register_columns('rsi')
    
    def _fetch_data(self, symbols, start_date, end_date, _timeframe):
        df = pd.read_csv('data/prices.csv')
        df = df[df['symbol'].isin(symbols)]
        df['date'] = pd.to_datetime(df['date'])
        return df[(df['date'] >= start_date) & (df['date'] <= end_date)]

The ```CSVDataSource``` reads ```prices.csv``` from disk into a [Pandas](https://pandas.pydata.org/) [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). And because ```prices.csv``` contains a custom ```rsi``` column that we want to use, we also register the column with **PyBroker** using [pybroker.register_columns](https://pybroker.com/en/latest/reference/pybroker.scope.html#pybroker.scope.register_columns).

Also note that the ```DataFrame``` returned by your ```DataSource``` is expected to contain the following columns: ```symbol```, ```date```, ```open```, ```high```, ```low```, and ```close```.

Now we can query the CSV data from an instance of ```CSVDataSource```:

In [2]:
csv_data_source = CSVDataSource()
df = csv_data_source.query(['MCD', 'NKE', 'DIS'], '6/1/2021', '12/1/2021')
df

Loading bar data...
Loaded bar data: 0:00:00 



,date,symbol,open,high,low,close,rsi
0,2021-06-01,DIS,180.179993,181.009995,178.740005,178.839996,46.321532
1,2021-06-01,MCD,235.979996,235.990005,232.740005,233.240005,46.522926
2,2021-06-01,NKE,137.850006,138.050003,134.210007,134.509995,53.308085
3,2021-06-02,DIS,179.039993,179.100006,176.929993,177.000000,42.635256
4,2021-06-02,MCD,233.970001,234.330002,232.809998,233.779999,48.051484
...,...,...,...,...,...,...,...
382,2021-11-30,MCD,247.380005,247.899994,243.949997,244.600006,40.461178
383,2021-11-30,NKE,168.789993,171.550003,167.529999,169.240005,51.505558
384,2021-12-01,DIS,146.699997,148.369995,142.039993,142.149994,16.677555
385,2021-12-01,MCD,245.759995,250.899994,244.110001,244.179993,39.853689


To use our custom ```DataSource``` in a backtest, we add it to a [Strategy](https://pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy) in the same way as with any other ```DataSource```:

In [3]:
from pybroker import Strategy

def buy_low_sell_high_rsi(ctx):
    pos = ctx.long_pos() 
    if not pos and ctx.rsi[-1] < 30:
        ctx.buy_shares = 100
    elif pos and ctx.rsi[-1] > 70:
        ctx.sell_shares = pos.shares

strategy = Strategy(csv_data_source, '6/1/2021', '12/1/2021')
strategy.add_execution(buy_low_sell_high_rsi, ['MCD', 'NKE', 'DIS'])
result = strategy.backtest(calc_bootstrap=False)
result.orders

Backtesting: 2021-06-01 00:00:00 to 2021-12-01 00:00:00

Loading bar data...
Loaded bar data: 0:00:00 

Test split: 2021-06-01 00:00:00 to 2021-12-01 00:00:00


100% (129 of 129) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:02


,date,symbol,type,limit_price,fill_price,shares
id,,,,,,
1,2021-09-21,NKE,buy,NaN,154.86,100
2,2021-11-04,NKE,sell,NaN,173.82,100
3,2021-11-16,DIS,buy,NaN,159.40,100


Note that by having registered the custom ```rsi``` column with **PyBroker**, it then becomes accessible from the [ExecContext](https://pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext) via ```ctx.rsi```.